In [1]:
import sys

sys.path.append("../")
# import time
# time.sleep(7200)

In [2]:
config = {
    "pretrained_model": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract",
    "tokenizer": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract",
    "max_seq_length": 512,
    "batch_size": 8,
    "lr": 2e-5,
    "epochs": 25,
    "device": "cuda",
    "gpu_ids": "3",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps": 1,
    "warmup_proportion": 0.1,
    "gradient_accumulation_steps": 1,
    "num_labels": 2,
    "is_multilabel": False,
    "valid_metric": "f1",
    "model_save_dir": "../checkpoints/bert_cased_512_bio_26k_20210315/",
    "patience": 4,
}

In [3]:
from beta_nlp.models.bert_cls import BertModel

cls = BertModel(config)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSeque

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [30]:
import pandas as pd

data_df = pd.read_csv("/home/zm324/workspace/data/med_cls/promed_26k/biocls_26k.csv")
data_df.docs = data_df.docs.apply(lambda x: x.replace("\n", ". ").replace("\r", ""))
data_df.labels = data_df.labels.apply(lambda x: "01" if x == 0 else "10")
data_df.head()

,Unnamed: 0,index,docs,labels,source,flag
0,0,32,India's Deccan gets more planes. . . . Air...,01,bbc,train
1,1,10431,. . DECLINES COMMENT ON HONGKONG LAND REPOR...,01,Reuters,train
2,2,1676,Letter to XPO staff reveals extra safety measu...,10,promed_extended,train
3,3,13747,. . . . KEYCORP <KEY> 3RD QTR NET. . . ...,01,Reuters,train
4,4,4435,. . . . API SAYS DISTILLATE STOCKS OFF 2.5...,01,Reuters,train


In [33]:
list(data_df.labels)[0]

'01'

In [5]:
len(data_df.index)

25956

In [7]:
train_set.to_csv(
    "../covid19-classification/hedwig-data/datasets/Promed_26k/train.tsv",
    columns=["labels", "docs"],
    header=False,
    index=False,
    sep="\t",
)
dev_set.to_csv(
    "../covid19-classification/hedwig-data/datasets/Promed_26k/dev.tsv",
    columns=["labels", "docs"],
    header=False,
    index=False,
    sep="\t",
)
test_set.to_csv(
    "../covid19-classification/hedwig-data/datasets/Promed_26k/test.tsv",
    columns=["labels", "docs"],
    header=False,
    index=False,
    sep="\t",
)

In [7]:
len(train_set.index), len(dev_set.index), len(test_set.index)

(20764, 2596, 2596)

In [8]:
train_set.groupby(["labels"]).size()

labels
01    17715
10     3049
dtype: int64

In [9]:
train_set.head()

,Unnamed: 0,index,docs,labels,source,flag
0,0,32,India's Deccan gets more planes. . . . Air...,01,bbc,train
1,1,10431,. . DECLINES COMMENT ON HONGKONG LAND REPOR...,01,Reuters,train
2,2,1676,Letter to XPO staff reveals extra safety measu...,10,promed_extended,train
3,3,13747,. . . . KEYCORP <KEY> 3RD QTR NET. . . ...,01,Reuters,train
4,4,4435,. . . . API SAYS DISTILLATE STOCKS OFF 2.5...,01,Reuters,train


In [19]:
import numpy as np

train_set = data_df[data_df["flag"] == "train"]
dev_set = data_df[data_df["flag"] == "valid"]
test_set = data_df[data_df["flag"] == "test"]
train_set.labels = train_set.labels.apply(lambda x: np.array([float(i) for i in x]))
dev_set.labels = dev_set.labels.apply(lambda x: np.array([float(i) for i in x]))
test_set.labels = test_set.labels.apply(lambda x: np.array([float(i) for i in x]))

In [20]:
cls.train(train_set, dev_set)

ValueError: Expected input batch_size (8) to match target batch_size (16).

In [ ]:
result = cls.test(test_set)
result

In [10]:
import csv

with open("../results/bert_cased_512_bio_26k_20200908_test.csv", "w") as f:
    for key in result.keys():
        f.write("%s,%s\n" % (key, result[key]))

In [17]:
isinstance(labels[0], (list))

False

In [28]:
data_dir = "/home/zm324/workspace/data/med_cls/promed_26k/"
filename = "biocls_26k.csv"
import os

df = pd.read_csv(os.path.join(data_dir, filename))

In [29]:
df.head()

,Unnamed: 0,index,docs,labels,source,flag
0,0,32,India's Deccan gets more planes\n \n \n \n Air...,0,bbc,train
1,1,10431,\n \n DECLINES COMMENT ON HONGKONG LAND REPOR...,0,Reuters,train
2,2,1676,Letter to XPO staff reveals extra safety measu...,1,promed_extended,train
3,3,13747,\n \n \r\n \n KEYCORP <KEY> 3RD QTR NET\r\n \n...,0,Reuters,train
4,4,4435,\n \n \r\n \n API SAYS DISTILLATE STOCKS OFF 2...,0,Reuters,train


In [37]:
import torch
torch.tensor([0,1]).argmax()

tensor(1)